In [1]:
import sys

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
#from keras import backend as K
from keras import optimizers
#K.set_image_dim_ordering('th')
# setting up a random seed for reproducibility
random_seed = 611
np.random.seed(random_seed)


Using TensorFlow backend.


AttributeError: module 'keras_preprocessing.text' has no attribute 'tokenizer_from_json'

In [ ]:
import numpy as np
import pandas as pd
from dl_utils import plot_metrics, plot_confusion_matrix
np.random.seed(42)

from IPython.display import display

# Importing libraries
from keras.models import Model, Sequential
from keras.layers import LSTM, BatchNormalization
from keras.layers import Dense, Dropout
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint

__LIST OF FETAURES and ACTIVITIES__

In [ ]:
SIGNALS = ["body_acc_x",
           "body_acc_y",
           "body_acc_z",
           "body_gyro_x",
           "body_gyro_y",
           "body_gyro_z",
           "total_acc_x",
           "total_acc_y",
           "total_acc_z"]

ACTIVITIES = {1 : 'WALKING',
              2 : 'WALKING_UPSTAIRS',
              3 : 'WALKING_DOWNSTAIRS',
              4 : 'SITTING',
              5 : 'STANDING',
              6 : 'LAYING'}

## 6.1 Loading the data

* Here we load the raw time-series data in which each datapoint has 128 readings each for 9 different signals.

In [ ]:
# Utility function to read the data from csv file
def read_txt(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

def load_data(subset, signals=SIGNALS):
    signals_data = []
    
    #signal_data
    for s in signals:
        #print(s)
        filename = f'UCI_HAR_Dataset/{subset}/Inertial Signals/{s}_{subset}.txt'
        signals_data.append(read_txt(filename).values)
        
    #label_data
    filename = f'UCI_HAR_Dataset/{subset}/y_{subset}.txt'
    y = read_txt(filename)[0]
    
    #reshape the signals_data into (#samples, 128readings along time, 9featueres)
    return np.transpose(signals_data, axes=(1, 2, 0)), pd.get_dummies(y).values

In [ ]:
Xtrain, Ytrain = load_data('train')
Xtest, Ytest = load_data('test')

print(f"Shape of Xtrain : {Xtrain.shape}")
print(f"Shape of Xtest  : {Xtest.shape}")

In [ ]:
print(f"Number of data_points in Xtrain               : {Xtrain.shape[0]}")
print(f"Number of data_points in Xtest                : {Xtest.shape[0]}")

print(f"Number of features/signals                    : {Xtrain.shape[-1]}")
print(f"Number of readings along time per each signal : {Xtrain.shape[1]}")

In [ ]:
def metrics(Xtrain, Ytrain, Xtest, Ytest, model):
    
    train = model.evaluate(Xtrain, Ytrain, verbose=0)
    test = model.evaluate(Xtest, Ytest, verbose=0)
    
    display(pd.DataFrame([train, test], columns=['LOG_LOSS', 'ACCURACY'], index=['TRAIN', 'TEST']))
    
    return train + test

In [ ]:
models_eval_dict = {'columns' : ['TRAIN_LOG_LOSS', 'TRAIN_ACC', "TEST_LOG_LOSS", "TEST_ACC"]}

## 6.1 Neural Net

* We'll use a __simple neural net with two hidden layers__ to train the model.
* The input will be the flattened 128 vectors of 9 signals stripping the time dependency.

In [ ]:
Xtrain_reshaped = Xtrain.reshape(Xtrain.shape[0], -1)
Xtest_reshaped = Xtest.reshape(Xtest.shape[0], -1)

print(f"Shape of Xtrain_reshaped : {Xtrain_reshaped.shape}")
print(f"Shape of Xtest_reshaped  : {Xtest_reshaped.shape}")

In [ ]:
model1 = Sequential()
model1.add(Dense(128, activation='relu', kernel_initializer='he_normal', 
                 input_shape=(Xtrain.reshape(Xtrain.shape[0], -1).shape[1:])))
model1.add(Dropout(0.25))
model1.add(Dense(64, activation='relu', kernel_initializer='he_normal'))
model1.add(Dropout(0.25))
model1.add(Dense(32, activation='relu', kernel_initializer='he_normal'))
model1.add(Dropout(0.25))
model1.add(Dense(Ytrain.shape[1], activation='softmax'))
model1.summary()

In [ ]:
# Compiling the model
model1.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

#checkpoint the best model based on log_loss
filepath = 'saved_models/dnn_best_weights2.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', save_best_only=True)
callbacks_list = [checkpoint]

In [ ]:
# Training the model
model1.fit(Xtrain_reshaped,
           Ytrain,
           batch_size=512,
           validation_data=(Xtest_reshaped, Ytest),
           epochs=75, 
           shuffle=True,
           callbacks=callbacks_list)

In [ ]:
plot_metrics(model1.history)

In [ ]:
model1.load_weights(filepath)

In [ ]:
models_eval_dict['DNN'] = metrics(Xtrain_reshaped, Ytrain, Xtest_reshaped, Ytest, model1)

In [ ]:
plot_confusion_matrix(Ytest.argmax(1), model1.predict(Xtest_reshaped).argmax(1), labels=ACTIVITIES.values())

__<font color='green'> OBSERVATIONS</font>__

* A simple neural net achieved good performance ~90% accuracy.
* The confusion matrix shows that the same issue exists as before <br>
with the classifier having a hard time to seperate STANDING and SITTING classes.
* The precision and recall values of classes : SITTING and STANDING are worse.
* The log loss(0.26) is significantly greater than the linear model's one(0.16).

## 6.2 LSTM

* We'll build an RNN with two LSTM layers stacked upon one other.

In [ ]:
# Initiliazing the sequential model
model2 = Sequential()
# Configuring the parameters
model2.add(LSTM(96, input_shape=(Xtrain.shape[1:]), return_sequences=True))
# Adding a dropout layer
model2.add(Dropout(0.25))

model2.add(LSTM(64))
# Adding a dropout layer
model2.add(Dropout(0.7))
# Adding a dense output layer with sigmoid activation
model2.add(Dense(Ytrain.shape[1], activation='softmax'))
model2.summary()

In [ ]:
#sgd = SGD(lr=0.02, decay=1e-6, momentum=0.9, nesterov=True, clipnorm=5)

# Compiling the model
model2.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

filepath = 'saved_models/lstm_best_weights.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', save_best_only=True)
callbacks_list = [checkpoint]

In [ ]:
# Training the model
model2.fit(Xtrain,
          Ytrain,
          batch_size=1024,
          validation_data=(Xtest, Ytest),
          epochs=100,
          shuffle=True,
          callbacks=callbacks_list)

In [ ]:
plot_metrics(model2.history)

In [ ]:
filepath = 'saved_models/lstm_best_weights.hdf5'
model2.load_weights(filepath)

In [ ]:
models_eval_dict['LSTM'] = metrics(Xtrain, Ytrain, Xtest, Ytest, model2)

In [ ]:
plot_confusion_matrix(Ytest.argmax(1), model2.predict(Xtest).argmax(1), labels=ACTIVITIES.values())

__<font color='green'> OBSERVATIONS</font>__

* Training a complex network like RNN requires huge computational resources and data.
* The same issue exists here. The performance is worser than the linear models.

## 6.3 Models Summary

In [ ]:
if 'columns' in models_eval_dict:
    columns = models_eval_dict.pop('columns')
    
display(pd.DataFrame.from_dict(models_eval_dict, orient='index', columns=columns))

## 6.4 Conclusion

* Both the deep neural nets (DNN and RNN) failed to overcome the limitations of the linear models.
* __It is to be noted that the models were trained on the raw time-series data__.
* Even a simple 1 layer neural net tends to perform on par with these models.
* Both the models showed signs of overfitting.
* After much  hyper-parameter tuning, it was found out that __we are beating a dead horse__ 
<br> and the performance never seemed to improve.